### Maximal Marginal Relevance
MMR (Maximal Marginal Relevance) is a powerful diversity-aware retrieval technique used in information retrieval and RAG pipelines to balance relevance and novelty when selecting documents.

In [1]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chat_models import init_chat_model
from langchain_classic.prompts import PromptTemplate
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains.retrieval import create_retrieval_chain


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [4]:
# Step 1: Load and chunk the document
loader = TextLoader("langchain_rag_dataset.txt")
raw_docs = loader.load()
splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap = 50
)

chunks = splitter.split_documents(raw_docs)
chunks

[Document(metadata={'source': 'langchain_rag_dataset.txt'}, page_content='LangChain is an open-source framework designed to simplify the development of applications using large language models (LLMs).\nLangChain provides abstractions for working with prompts, chains, memory, and agents, making it easier to build complex LLM-based systems.'),
 Document(metadata={'source': 'langchain_rag_dataset.txt'}, page_content='The framework supports integration with various vector databases like FAISS and Chroma for semantic retrieval.\nLangChain enables Retrieval-Augmented Generation (RAG) by allowing developers to fetch relevant context before generating responses.'),
 Document(metadata={'source': 'langchain_rag_dataset.txt'}, page_content='Memory in LangChain helps models retain previous interactions, making multi-turn conversations more coherent.\nAgents in LangChain can use tools like calculators, search APIs, or custom functions based on the instructions they receive.'),
 Document(metadata={'

In [5]:
# Step 2: FAISS Vector Store with HuggingFace Embeddings
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vector_store =  FAISS.from_documents(
    documents=chunks,
    embedding=embedding_model
)

In [6]:
### Step 3: Create MMR Retirever

retriver = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k":3}
)


In [8]:
# Step 4: Prompt and LLM

prompt = PromptTemplate.from_template(
"""
Answer the question based on the context provided.

Context:
{context}

Question: {input}
"""
)

llm = init_chat_model("groq:llama-3.1-8b-instant")

In [9]:
# Step 5: RAG Pipeline
document_chain = create_stuff_documents_chain(
    llm,
    prompt
)

rag_chain = create_retrieval_chain(
    retriever=retriver,
    combine_docs_chain=document_chain
)

In [10]:
# Step 6: Query
query = {"input": "How does LangChain support agents and memory?"}

response = rag_chain.invoke(query)

print("✅ Answer:\n", response["answer"])

✅ Answer:
 LangChain supports agents and memory in the following ways:

1. **Agents**: LangChain allows agents to use tools like calculators, search APIs, or custom functions based on the instructions they receive. Agents can also interact with external APIs and databases, enhancing the capabilities of LLM-powered applications. Additionally, LangChain enables agents to act as LLMs that decide which tool to call and in what order during a task.

2. **Memory**: LangChain supports conversational memory using tools like `ConversationBufferMemory` and summarization memory with `ConversationSummaryMemory`. This helps models retain previous interactions, making multi-turn conversations more coherent.


In [11]:
response

{'input': 'How does LangChain support agents and memory?',
 'context': [Document(id='8020a27b-6ee0-42da-b2fb-ee4c23c42441', metadata={'source': 'langchain_rag_dataset.txt'}, page_content='Memory in LangChain helps models retain previous interactions, making multi-turn conversations more coherent.\nAgents in LangChain can use tools like calculators, search APIs, or custom functions based on the instructions they receive.'),
  Document(id='34de8504-a4b1-4055-ac76-6a8dc8d344b8', metadata={'source': 'langchain_rag_dataset.txt'}, page_content='LangChain agents can interact with external APIs and databases, enhancing the capabilities of LLM-powered applications.\nRAG pipelines in LangChain involve document loading, splitting, embedding, retrieval, and LLM-based response generation.'),
  Document(id='e3629447-fa40-43ba-9c66-add241c7c486', metadata={'source': 'langchain_rag_dataset.txt'}, page_content='LangChain allows LLMs to act as agents that decide which tool to call and in what order duri